## API key

### Get one

To run this code, you need an API key from Open AI. This involves giving them your credit card and setting up spending limits. 

### Using it

I run this file locally via Jupyterlab, so it's in a folder with `gpt_api.txt` which contains my API key. 

To run this file in Google Colab, you _could_ directly type your API key into the notebook below, **but this is a bad idea.** 

Instead, one common way is to store the API key in a file on your Google Drive and then access it from the Colab notebook. Here's how you can do it:

1.    Create a new text file on your Google Drive and store your API key in it. Name the file something like `gpt_api.txt`.
1.    Mount your Google Drive to the Google Colab notebook by running the following code block.
    ```python
    import openai
    from google.colab import drive
    drive.mount('/content/drive')
    with open('/content/drive/gpt_api.txt', 'r') as f:
        openai.api_key = f.read().strip()
    ```
1.     This will prompt you to click on a link to authorize the connection. Follow the instructions, and copy the authorization code into the input box that appears in the Colab notebook. You can now continue on. 

In [1]:
# !pip install openai 

In [4]:
import openai

# don't type the key in this file! open it from file that is in gitignore, github secrets, or in your google drive

with open('gpt_api.txt', 'r') as f:
    openai.api_key = f.read().strip()

## Define key functions to do the lift

In [13]:
# gpt 4.0 wrote this mostly

import os
import glob

import numpy as np
import pandas as pd
from IPython.display import (  # used during dev - display(Markdown(markdown_table)) prints nice
    Markdown,
    display,
)
from tqdm import tqdm
from bs4 import BeautifulSoup

# Set Pandas display options to show full string
pd.set_option("display.max_colwidth", None)

def ask_openai(question, data):
    prompt = f"{data}\n---\n{question}"
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=70,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].text.strip()

def parse_file(filename):

    # Define your question related to the loan application
    question = "Output a tab separated list containing two items: the name of the buyer, and the name of the seller."

    # remove the html
    with open(filename, "r") as fp:
        raw = BeautifulSoup(fp.read(), 'html.parser').get_text()

    return ask_openai(question, raw[:1850])

In [14]:
file_sentence_dict = {}
files = glob.glob("inputs/*") #get all the files in the inputs folder

for file in tqdm(files,total=len(files)):
    file_sentence_dict.update({file: parse_file(file)}) #update the dictionary 

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.58it/s]


In [15]:
file_sentence_dict

{'inputs\\ex10-11.txt': 'Baxter Healthcare Corporation\tCFC International, Inc.',
 'inputs\\ex10.txt': 'Baxter Healthcare Corporation\tCFC International'}

## Examine output

In [18]:
df = pd.DataFrame(file_sentence_dict.items(), columns=['document', 'buyer_seller'])
df[['buyer', 'seller']] = df['buyer_seller'].str.split('\t', expand=True)
df = df.drop('buyer_seller', axis=1)
df


,document,buyer,seller
0,inputs\ex10-11.txt,Baxter Healthcare Corporation,"CFC International, Inc."
1,inputs\ex10.txt,Baxter Healthcare Corporation,CFC International
